In [20]:
import numpy as np
import pandas as pd
import xarray as xr
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/pbig5min.dat' #Ubuntu
path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/pbig1min.dat' #Server

#read the REPPU data
with open (path) as f:
    rectype = np.dtype(np.float32)
    reppu_data = np.fromfile(f, rectype) #size = 109,900,800

reppu_data = reppu_data.reshape(-1,30,80)
reppu_data.shape

(228960, 30, 80)

In [5]:
#Read MHD dates and expand-out the date ranges
mhd_data = pd.read_csv('mhd_dates.csv')
expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
    for start, end in zip(mhd_data['start'], mhd_data['end'])])

In [6]:
time_res = 1

if time_res == 1:
    time_step = 1440
else :
    time_step = 288

In [7]:
data_reshaped = reppu_data.reshape(len(expanded_dt), time_step, 30, 80) 

# Define coordinates
time = np.arange(time_step)
lat = np.linspace(53.1, 89.7, 30) #30 intervals between 53.1° to 89.7°
lon = np.linspace(1.6, 357.6, 80) #80 intervals between 1.6° to 357.6°

# Create 'dt' variable combining dates and five-minute intervals
dt = []
for day in expanded_dt:
    for t in time:
        dt.append(day + pd.Timedelta(minutes=t*time_res))
dt = np.array(dt) #convert from list to numpy array

# Create xarray Dataset
ds = xr.Dataset({'potential': (['dt', 'lat', 'lon'], data_reshaped.reshape(-1, 30, 80))},
coords={'dt': dt, 'lat': lat, 'lon': lon})

# Add potential and units
ds['potential'].attrs['units'] = 'kV'
ds['potential'] = ds['potential'] * 1e-3 # Convert to kV
ds

<xarray.Dataset>
Dimensions:    (dt: 228960, lat: 30, lon: 80)
Coordinates:
  * dt         (dt) datetime64[ns] 2021-05-10 ... 2022-08-19T23:59:00
  * lat        (lat) float64 53.1 54.36 55.62 56.89 ... 85.91 87.18 88.44 89.7
  * lon        (lon) float64 1.6 6.106 10.61 15.12 ... 344.1 348.6 353.1 357.6
Data variables:
    potential  (dt, lat, lon) float32 1.279 1.026 0.7595 ... -23.67 -23.72

In [8]:
omni_df = pd.read_csv('omni_mhd_5min.csv')
#omni_df = pd.read_csv(omni_mhd_path+'omni_mhd_5min.csv')
omni_df.set_index('dt', inplace=True) #set the datetime as the index
omni_df = omni_df.ffill().bfill()
omni_df = omni_df.dropna() #drop any remaining NaNs
omni_df

omni_ds = xr.Dataset(omni_df)
omni_ds['dt'] = pd.to_datetime(omni_ds['dt']) #convert the index to datetime

#merge OMNI with REPPU data
reppu_omni_ds = ds.merge(omni_ds, join='outer')

#select date range
reppu_omni_ds = reppu_omni_ds.sortby('dt')
reppu_omni_ds

#interpolate the missing values in BY_GSE
reppu_omni_ds['BY_GSE'] = reppu_omni_ds['BY_GSE'].interpolate_na(dim='dt')
reppu_omni_ds['BZ_GSE'] = reppu_omni_ds['BZ_GSE'].interpolate_na(dim='dt')
reppu_omni_ds['flow_speed'] = reppu_omni_ds['flow_speed'].interpolate_na(dim='dt')
reppu_omni_ds['proton_density'] = reppu_omni_ds['proton_density'].interpolate_na(dim='dt')
reppu_omni_ds['tilt_angle'] = reppu_omni_ds['tilt_angle'].interpolate_na(dim='dt')

#drop nan values
reppu_omni_ds = reppu_omni_ds.dropna('dt')
reppu_omni_ds

<xarray.Dataset>
Dimensions:         (dt: 228956, lat: 30, lon: 80)
Coordinates:
  * dt              (dt) datetime64[ns] 2021-05-10 ... 2022-08-19T23:55:00
  * lat             (lat) float64 53.1 54.36 55.62 56.89 ... 87.18 88.44 89.7
  * lon             (lon) float64 1.6 6.106 10.61 15.12 ... 348.6 353.1 357.6
Data variables:
    potential       (dt, lat, lon) float32 1.279 1.026 0.7595 ... -23.61 -23.65
    BY_GSE          (dt) float64 4.72 4.646 4.572 4.498 ... -1.968 -1.944 -1.92
    BZ_GSE          (dt) float64 6.24 6.302 6.364 6.426 ... -4.556 -4.528 -4.5
    flow_speed      (dt) float64 344.0 344.0 344.0 344.0 ... 656.0 657.0 658.0
    proton_density  (dt) float64 12.11 11.98 11.84 11.71 ... 5.49 5.44 5.39 5.34
    tilt_angle      (dt) float64 0.253 0.2524 0.2517 ... 0.1748 0.1742 0.1735

In [27]:
X_train_a = reppu_omni_ds.sel(dt=slice("2021-05-10","2022-06-30")) #123 days
X_train_b = reppu_omni_ds.sel(dt=slice("2022-08-15","2022-08-19")) #5 days
X_train_ds = xr.concat([X_train_a, X_train_b], dim='dt') #128 days ~ 80% of the data
X_test_ds = reppu_omni_ds.sel(dt=slice("2022-07-01","2022-07-31")) #31 days ~ 20% of the data
X_test_unscaled_ds = X_test.copy() #keep a copy of the unscaled test data for later use

In [28]:
# List of variables to be scaled (excluding 'potential')
variables_to_scale = [var for var in X_train_ds.data_vars if var != 'potential']

# Fit the scaler on training data
scaler = MinMaxScaler()

for var in variables_to_scale:
    # Reshape the variable to (-1, 1) to fit the scaler
    X_train_values = X_train_ds[var].values.reshape(-1, 1)
    X_test_values = X_test_ds[var].values.reshape(-1, 1)

    # Fit and transform on training data, then transform on test data
    scaled_train_values = scaler.fit_transform(X_train_values).reshape(X_train_ds[var].shape)
    scaled_test_values = scaler.transform(X_test_values).reshape(X_test_ds[var].shape)
    
    # Update the xarray dataset with the scaled values
    X_train_ds[var] = (X_train_ds[var].dims, scaled_train_values)
    X_test_ds[var] = (X_test_ds[var].dims, scaled_test_values)


In [32]:
X_train_ds

<xarray.Dataset>
Dimensions:         (dt: 184316, lat: 30, lon: 80)
Coordinates:
  * dt              (dt) datetime64[ns] 2021-05-10 ... 2022-08-19T23:55:00
  * lat             (lat) float64 53.1 54.36 55.62 56.89 ... 87.18 88.44 89.7
  * lon             (lon) float64 1.6 6.106 10.61 15.12 ... 348.6 353.1 357.6
Data variables:
    potential       (dt, lat, lon) float32 1.279 1.026 0.7595 ... -23.61 -23.65
    BY_GSE          (dt) float64 0.606 0.6045 0.6029 ... 0.4656 0.4661 0.4666
    BZ_GSE          (dt) float64 0.615 0.6163 0.6176 ... 0.3915 0.3921 0.3927
    flow_speed      (dt) float64 0.1137 0.1137 0.1137 ... 0.6769 0.6787 0.6805
    proton_density  (dt) float64 0.1902 0.188 0.1858 ... 0.08009 0.07926 0.07843
    tilt_angle      (dt) float64 0.7216 0.721 0.7205 ... 0.6532 0.6526 0.652

In [61]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        # Remove extra dimensions from output
        out = self.fc(out[:, -1, :].squeeze())  # Squeeze the output
        return out

    def reset_states(self):
        # Reset the internal states of the LSTM layer
        self.lstm.reset_parameters()

In [62]:
seq_len = 30 # Number of time steps to look back 
slider = 5 # Number of time steps to slide the window forward

# Instantiate the LSTM model with updated input_size
input_size = 5
hidden_size = 64
num_layers = 2
output_size = 2400
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [64]:
mhd_dates = [
    {"start": "2021/05/10", "end": "2021/05/14", "total_days": 5},
    {"start": "2021/05/31", "end": "2021/06/03", "total_days": 4},
    {"start": "2021/07/26", "end": "2021/07/29", "total_days": 4},
    {"start": "2021/09/09", "end": "2021/09/12", "total_days": 4},
    {"start": "2021/10/11", "end": "2021/10/14", "total_days": 4},
    {"start": "2021/11/01", "end": "2021/11/06", "total_days": 6},
    {"start": "2021/11/25", "end": "2021/11/29", "total_days": 5},
    {"start": "2021/12/01", "end": "2022/01/24", "total_days": 55},
    {"start": "2022/01/30", "end": "2022/02/03", "total_days": 5},
    {"start": "2022/03/11", "end": "2022/03/15", "total_days": 5},
    {"start": "2022/03/28", "end": "2022/04/01", "total_days": 5},
    {"start": "2022/06/10", "end": "2022/06/30", "total_days": 20},
    #{"start": "2022/07/01", "end": "2022/07/31", "total_days": 31}, #test data
    #{"start": "2022/08/15", "end": "2022/08/19", "total_days": 5}
]

for date_range in mhd_dates:
    start_date = datetime.datetime.strptime(date_range["start"], "%Y/%m/%d")
    end_date = datetime.datetime.strptime(date_range["end"], "%Y/%m/%d")
    #total_days = date_range["total_days"]

    X_train_ds_loop = X_train_ds.sel(dt=slice(start_date, end_date))
    X_train = X_train_ds_loop.drop('potential').to_array().values.T
    X_train = X_train[:-1,:] #remove extra var

    y_train = X_train_ds_loop['potential'].values
    y_train = y_train.reshape(-1, 30*80)
    y_train = y_train[:-1,:] #remove extra var
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

    num_epochs = 1
    model.reset_states()
    for epoch in range(num_epochs):
        for i in range(0, len(X_train_tensor) - seq_len + 1, slider):
            inputs = X_train_tensor[i:i + seq_len].to(device)
            targets = y_train_tensor[i + seq_len - 1].to(device)
            # Forward pass
            outputs = model(inputs.unsqueeze(0)) # Add batch dimension
            loss = criterion(outputs, targets)
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        rmse = np.sqrt(loss.item())
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.2f}, RMSE: {rmse:.2f} V')

Epoch [1/1], Loss: 89.02, RMSE: 9.43 V
Epoch [1/1], Loss: 0.24, RMSE: 0.49 V
Epoch [1/1], Loss: 34.63, RMSE: 5.88 V
Epoch [1/1], Loss: 36.14, RMSE: 6.01 V
Epoch [1/1], Loss: 14.15, RMSE: 3.76 V
Epoch [1/1], Loss: 19.60, RMSE: 4.43 V
Epoch [1/1], Loss: 72.38, RMSE: 8.51 V
Epoch [1/1], Loss: 12.10, RMSE: 3.48 V
Epoch [1/1], Loss: 199.74, RMSE: 14.13 V
Epoch [1/1], Loss: 87.05, RMSE: 9.33 V
Epoch [1/1], Loss: 66.86, RMSE: 8.18 V
Epoch [1/1], Loss: 43.25, RMSE: 6.58 V
